In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [ ]:
!pip install indic-nlp-library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 40 kB 3.1 MB/s 
     |████████████████████████████████| 2.8 MB 11.1 MB/s 


In [ ]:
df = pd.read_csv('/content/tamil_offensive_train (1).tsv',sep = '\t')
print(df)

           id                                               text category
0        tam1  திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...      NOT
1        tam2  இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...      OFF
2        tam3  மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...      NOT
3        tam4  மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......      OFF
4        tam5                     only for விஜய் சேதுபதி and STR      NOT
...       ...                                                ...      ...
5875  tam5876  இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...      NOT
5876  tam5877  இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...      NOT
5877  tam5878  கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...      NOT
5878  tam5879  அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....      OFF
5879  tam5880  உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...      NOT

[5880 rows x 3 columns]


In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 574 kB 5.3 MB/s 
     |████████████████████████████████| 175 kB 53.9 MB/s 
     |████████████████████████████████| 4.4 MB 46.0 MB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
     |████████████████████████████████| 6.6 MB 32.1 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=518501ed4c5ef87de0e2c981ab36387b85c0f1afc4db49a5883a6c11e369427d
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import stanza

In [ ]:
stanza.download('ta')

2022-06-25 08:28:11 INFO: Downloading default packages for language: ta (Tamil)...


2022-06-25 08:28:21 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp = stanza.Pipeline(lang='ta', processors='tokenize,mwt,pos,lemma')

2022-06-25 08:28:22 INFO: Loading these models for language: ta (Tamil):
| Processor | Package |
-----------------------
| tokenize  | ttb     |
| mwt       | ttb     |
| pos       | ttb     |
| lemma     | ttb     |

2022-06-25 08:28:22 INFO: Use device: cpu
2022-06-25 08:28:22 INFO: Loading: tokenize
2022-06-25 08:28:22 INFO: Loading: mwt
2022-06-25 08:28:22 INFO: Loading: pos
2022-06-25 08:28:23 INFO: Loading: lemma
2022-06-25 08:28:23 INFO: Done loading processors!


In [ ]:
def lemmatize_text(str):
  doc= nlp(str)
  List= {word.lemma for sent in doc.sentences for word in sent.words }
  s= ' '
  s = s.join(List)
  return s

In [ ]:
df['text'] = df.text.apply(lemmatize_text)

In [ ]:
print(df['text'][0])

வெற்றி வாழ் பெறு திருமலை பேரவை படம் சார்பு நாயக்கர்


In [ ]:
def zeroorone(s):
  if(s == 'NOT'):
    return 0
  elif(s=='OFF'):
    return 1
  else :
    return 0

In [ ]:
df['category'] = df.category.apply(zeroorone)

In [ ]:
from urllib import response
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
text = df['text'].values

In [ ]:
import joblib
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [ ]:
response = tfidf.fit_transform(text)

In [ ]:
joblib.dump(tfidf,'tfidf.pkl')

['tfidf.pkl']

In [ ]:
response.shape

(5880, 3339)

In [ ]:
y = df.category.values

In [ ]:
reg = linear_model.LogisticRegression(C=200.0,penalty= 'l2',solver = 'saga',multi_class = 'ovr', warm_start = True)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(response, y,test_size = 0.2, random_state = 4)

In [ ]:
reg.fit(response,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(C=200.0, multi_class='ovr', solver='saga', warm_start=True)

In [ ]:
joblib.dump(reg,'linearmodel.pkl')

['linearmodel.pkl']

In [ ]:
reg.score(x_test,y_test)

0.907312925170068

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
df2 = pd.read_csv('./tamil_offensive_test task 1 with labels.tsv',sep = '\t')

In [ ]:
df2['category'] = df2.category.apply(zeroorone)

In [ ]:
df2['text'] = df2.text.apply(lemmatize_text)

In [ ]:
#tfidf_from_joblib = joblib.load('tfidf.pkl')

In [ ]:
#reg_from_joblib = joblib.load('linearmodel.pkl')

In [ ]:
text1 = df2['text'].values
response1 = tfidf.transform(text1)

In [ ]:
response1.shape

(654, 3339)

In [ ]:
y1 = df2.category.values

In [ ]:
pred = reg.predict(response1)
print(classification_report(y1, pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.89       536
           1       0.47      0.39      0.43       118

    accuracy                           0.81       654
   macro avg       0.67      0.65      0.66       654
weighted avg       0.80      0.81      0.80       654



using keras on vectors

In [ ]:
pip install tensorflow

In [ ]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


using knn 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(response, y,test_size = 0.2, random_state = 4)

In [ ]:
knn.fit(response, y)

KNeighborsClassifier(n_neighbors=2)

In [ ]:
joblib.dump(knn,'knn.pkl')

['knn.pkl']

In [ ]:
y_predict = knn.predict(response1)

In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.82      0.98      0.89       536
           1       0.23      0.03      0.05       118

    accuracy                           0.81       654
   macro avg       0.53      0.50      0.47       654
weighted avg       0.71      0.81      0.74       654



using svm

In [ ]:
import numpy as np

In [ ]:
# x = np.asarray(response)
y= np.asarray(y)


In [ ]:
from sklearn.datasets import make_classification
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid  = { 'kernel' : ('linear' , 'poly', 'rbf', 'sigmoid', 'precomputed') , 'C': [1, 10, 100] , 'degree' : [1,2,3,4,5], 'gamma' :['scale', 'auto', 1.0 , 10.0 , 100.0] , 'coef0' : [0.0,0.5,0.05,1,2.0,6.0,10.0,50.0] }

In [ ]:
from sklearn import svm
# classifier = svm.SVC(random_state = 4)
# x_train, x_test , y_train , y_test = train_test_split(response,y,test_size = 0.2,random_state = 4)

# sh = GridSearchCV(classifier , param_grid , cv = 5 ).fit(x_train,y_train)



In [ ]:
# sh.best_estimator_

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train , y_test = train_test_split(response,y,test_size = 0.2,random_state = 4)
classifier = svm.SVC(kernel = 'poly' , C = 100, degree = 2 , gamma = 'scale', coef0 = 1.0, random_state = 4)
classifier.fit(response,y)

SVC(C=100, coef0=1.0, degree=2, kernel='poly', random_state=4)

In [ ]:
joblib.dump(classifier,'svm.pkl')

['svm.pkl']

In [ ]:
y_predict = classifier.predict(response1)

In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89       536
           1       0.48      0.36      0.42       118

    accuracy                           0.81       654
   macro avg       0.68      0.64      0.65       654
weighted avg       0.80      0.81      0.80       654



using decision tree

In [ ]:
from sklearn import tree

In [ ]:
model = tree.DecisionTreeClassifier(splitter = 'random',min_samples_split = 3, random_state = 4, criterion = 'gini')

In [ ]:
model.fit(response,y)

DecisionTreeClassifier(min_samples_split=3, random_state=4, splitter='random')

In [ ]:
joblib.dump(model,'decisiontree.pkl')

['decisiontree.pkl']

In [ ]:
y_predict = model.predict(response1)

In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.87      0.90      0.89       536
           1       0.48      0.41      0.44       118

    accuracy                           0.81       654
   macro avg       0.68      0.65      0.66       654
weighted avg       0.80      0.81      0.81       654



using multinomial naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB(alpha = 0.0, fit_prior = False)

In [ ]:
clf.fit(response,y)

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


MultinomialNB(alpha=0.0, fit_prior=False)

In [ ]:
joblib.dump(clf,'mnb.pkl')

['mnb.pkl']

In [ ]:
y_predict = clf.predict(response1)

In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.89      0.76      0.82       536
           1       0.34      0.56      0.42       118

    accuracy                           0.72       654
   macro avg       0.61      0.66      0.62       654
weighted avg       0.79      0.72      0.75       654



using random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=50 , min_samples_split = 4, min_samples_leaf = 1 ,warm_start = True ,random_state = 4, class_weight= 'balanced_subsample', verbose = 10, n_jobs = 5)
model.fit(response,y)
joblib.dump(model,'rf.pkl')

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:766: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  'class_weight presets "balanced" or '
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.1s


building tree 6 of 50building tree 7 of 50

building tree 8 of 50building tree 9 of 50building tree 10 of 50


building tree 11 of 50
building tree 12 of 50
building tree 13 of 50building tree 14 of 50

building tree 15 of 50
building tree 16 of 50
building tree 17 of 50building tree 18 of 50

building tree 19 of 50
building tree 20 of 50
building tree 21 of 50


[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    0.4s


building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50building tree 29 of 50

building tree 30 of 50
building tree 31 of 50


[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.6s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    0.7s


building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50building tree 39 of 50

building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50building tree 45 of 50

building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    1.0s
[Parallel(n_jobs=5)]: Done  47 out of  50 | elapsed:    1.1s remaining:    0.1s
[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed:    1.2s finished


['rf.pkl']

In [ ]:
y_predict = model.predict(response1)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  47 out of  50 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed:    0.0s finished


In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.85      0.97      0.91       536
           1       0.64      0.21      0.32       118

    accuracy                           0.84       654
   macro avg       0.74      0.59      0.61       654
weighted avg       0.81      0.84      0.80       654



using dirty_cat

In [ ]:
pip install dirty_cat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [ ]:
from dirty_cat import SuperVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

In [ ]:
pipeline = make_pipeline(SuperVectorizer(), RandomForestClassifier(n_estimators=50 , min_samples_split = 4, min_samples_leaf = 1 ,warm_start = True ,random_state = 4, class_weight= 'balanced_subsample', verbose = 10, n_jobs = 5))

In [ ]:
pipeline.fit(text , y)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:766: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  'class_weight presets "balanced" or '
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.1s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50building tree 9 of 50

building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50


[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    0.4s


building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50building tree 27 of 50

building tree 28 of 50
building tree 29 of 50
building tree 30 of 50building tree 31 of 50



[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.5s


building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50building tree 39 of 50

building tree 40 of 50
building tree 41 of 50


[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.9s


building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50building tree 47 of 50

building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=5)]: Done  47 out of  50 | elapsed:    1.1s remaining:    0.1s
[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed:    1.1s finished


Pipeline(steps=[('supervectorizer', SuperVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced_subsample',
                                        min_samples_split=4, n_estimators=50,
                                        n_jobs=5, random_state=4, verbose=10,
                                        warm_start=True))])

In [ ]:
y_predict = pipeline.predict(text1)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  47 out of  50 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done  50 out of  50 | elapsed:    0.0s finished


In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90       536
           1       0.58      0.31      0.41       118

    accuracy                           0.83       654
   macro avg       0.72      0.63      0.66       654
weighted avg       0.81      0.83      0.81       654



using pipeline in lr

In [ ]:
pipeline1 = make_pipeline(SuperVectorizer(), linear_model.LogisticRegression(C=200.0,penalty= 'l2',solver = 'saga',multi_class = 'ovr', warm_start = True) )

In [ ]:
pipeline1.fit(text , y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


Pipeline(steps=[('supervectorizer', SuperVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=200.0, multi_class='ovr', solver='saga',
                                    warm_start=True))])

In [ ]:
y_predict = pipeline1.predict(text1)

In [ ]:
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       536
           1       0.56      0.19      0.28       118

    accuracy                           0.83       654
   macro avg       0.70      0.58      0.59       654
weighted avg       0.79      0.83      0.79       654



svm using pipeline

In [ ]:
pipeline = make_pipeline(SuperVectorizer(),tree.DecisionTreeClassifier(splitter = 'random',min_samples_split = 3, random_state = 4, criterion = 'gini'))

In [ ]:
pipeline.fit(text , y)
y_predict = pipeline.predict(text1)
print(classification_report(y1, y_predict))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85       536
           1       0.37      0.48      0.42       118

    accuracy                           0.76       654
   macro avg       0.62      0.65      0.63       654
weighted avg       0.79      0.76      0.77       654



using xbgclassifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgbc = XGBClassifier(max_delta_step=10.0, max_depth=20,
              n_estimators=100)

In [ ]:
xgbc.fit(response,y)

In [ ]:
y_predict = xgbc.predict(response1)

In [ ]:
print(classification_report(y1, y_predict))

In [ ]:
param_grid = [{ 'n_estimators' : [100 , 120 , 90] , 'max_depth' : [15 , 20 , 17 ,22] , 'min_child_weight' : [1.5 , 1.8 , 2.0 , 2.2 , 2.4] , 'max_delta_step' : [9.5 , 9.8 , 10.0 , 10.5 , 11.0]}]

In [ ]:
from sklearn.metrics import f1_score, make_scorer
fone_score = f1_score

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV( xgbc , param_grid , cv =3 , scoring = 'accuracy', return_train_score= True , verbose = 10 )

In [ ]:
grid_search.fit(response , y)

In [ ]:
final_model = grid_search.best_estimator_

In [ ]:
final_model

In [ ]:
y_predict = final_model.predict(response1)

In [ ]:
print(classification_report(y1, y_predict))